### Install python modules
Install on SageMaker the necesary modules for this notebook to work

In [1]:
!/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip
!pip install pytube
!pip install seaborn
!pip install smart_open

  Using cached pip-21.0.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.3.3
    Uninstalling pip-20.3.3:
      Successfully uninstalled pip-20.3.3
     |████████████████████████████████| 41 kB 130 kB/s  eta 0:00:01
     |████████████████████████████████| 119 kB 16.3 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.2.0-py3-none-any.whl size=109630 sha256=99bc14ac66069a23871c52d3d1b371624353ff77bf972dc0323fc8bc99d6aec2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/05/12/87/d479d6a8f92130cd8b27e331cc433bb28dda9c20e57f0b1ab2
Successfully built smart-open


### Import modules

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import random as rd
from adaptors import youtube2df, aws2df, upload_yt_file, get_folder_list, vocabulary_shuffle
import logging
from pytube import YouTube
import boto3
from botocore.exceptions import ClientError
import sys, os
from urllib.parse import urlparse
import re

Define a YouTube video URL

In [4]:
url = 'https://youtu.be/eEOhx-u9Z2k'
#url = 'https://youtu.be/Mxv9AM397Y8'

# Get video object
yt = YouTube(url)

### Download videos from Youtube
We'll download videos from youtube, save them into an S3 bucket that triggers a transcribe job (with or without IPA vocabulary)

In [7]:
# Get the tittle and replace spaces with underscore
yt_title = yt.title.replace(' ', '_')
    
# Download the video to tmp folder and save the output name to file_name
file_name = yt.streams.first().download(output_path="/tmp" ,filename=yt_title)


In [8]:
# Returns True if file was uploaded, else False
upload_yt_file(file_name)

True

### Download captions from the YouTube video
From the said YouTube video we'll get the 'a.es' caption if exist 

In [11]:
if yt.captions.get_by_language_code('es-419'):
    code = 'es-419'
    caption = yt.captions.get_by_language_code('es-419')
    yt_caption = caption.generate_srt_captions()
    yt_title = yt.title.replace(' ', '_')
    with open(f'/tmp/{yt_title}_{code}.txt', 'a') as f:
        f.write(yt_caption)
else:
    print('This caption doesn\'t exist')
    print('You can use one of the following captions: \n', yt.captions.all())

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Call to deprecated function get_by_language_code (This object can be treated as a dictionary, i.e. captions['en']).
  app.launch_new_instance()


### Upload the caption to S3

In [12]:
upload_yt_file(f'/tmp/{yt_title}_{code}.txt', object_name=f'levenshteinTests/ytCaptions/{yt_title}.txt')

True

# Test and compare Data

In [6]:
aws_transcribe = aws2df('s3://awstranscribe-tests/levenshteinTests/RAW/utubeVideos/Un_show_FALLIDO_-_Mini_Especial_Rancagua__Edo_Caroe.json')
aws_transcribe.head()

,start,end,transcript
0,0.04,0.51,La
1,0.51,1.03,Fallida
2,1.03,1.73,actuación
3,1.74,1.92,en
4,1.92,3.08,Rancagua


In [5]:
yt_transcribe = youtube2df('s3://awstranscribe-tests/levenshteinTests/ytCaptions/Un_show_FALLIDO_-_Mini_Especial_Rancagua_|_Edo_Caroe.txt')
yt_transcribe.head()

,orig_index,start,end,transcript
0,1,0.00,5.43,la fallida actuación en rancagua 1
1,2,3.27,6.87,problema de este lugar web sin duda uno
2,3,5.43,9.09,de los shows más raros de la gira moto
3,4,6.87,15.99,al sur y probablemente uno de los
4,5,9.09,18.15,mejores pero bueno los shows de la gira


# IPA Shuffle Dictionary
From our universe of 124 words we'll randomly select N number of words and create a new dictionary: IPA_Shuffle

In [3]:
vocabulary_shuffle()

{'VocabularyName': 'IPA_Shuffle',
 'LanguageCode': 'es-ES',
 'VocabularyState': 'PENDING',
 'ResponseMetadata': {'RequestId': 'a8ac87e1-5385-476e-94b1-a0a51c23e92e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 02 Mar 2021 18:59:25 GMT',
   'x-amzn-requestid': 'a8ac87e1-5385-476e-94b1-a0a51c23e92e',
   'content-length': '83',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

# Troubleshooting different Functions
I'll use the next cells for debug